In [ ]:
import pandas as pd             
import sqlite3                  
from sklearn.model_selection import train_test_split  
from sklearn.ensemble import RandomForestClassifier   # Importation du classificateur RandomForest
from sklearn.metrics import classification_report, accuracy_score  # Importation des fonctions d'évaluation du modèle
from sklearn.preprocessing import LabelEncoder


In [ ]:
# Connexion à la base de données SQLite
conn = sqlite3.connect('../Data/db/fraude_detection_warehouse_.db')


In [ ]:
# Étape 1: Chargement les données de la table transactions
data = pd.read_sql("SELECT * FROM transactions", conn)


In [ ]:
X = data.drop('is_fraud', axis=1)   # Suppression de la colonne is_fraud' pour obtenir les caractéristiques
y = data['is_fraud'] # La cible est la colonne 'is_fraud'
y


In [ ]:
# Conversion de la colonne 'transaction_date' en type datetime
X['transaction_date'] = pd.to_datetime(X['transaction_date'], errors='coerce')

# Extraire jour, mois, et année de la colonne 'transaction_date'
X['jour'] = X['transaction_date'].dt.day
X['mois'] = X['transaction_date'].dt.month
X['annee'] = X['transaction_date'].dt.year

# Supprimer la colonne 'transaction_date'
X = X.drop(columns=['transaction_date'])

# Vérifier les changements
print(X.dtypes)


In [ ]:
# Supprimer les colonnes ID qui ne sont pas des features pertinentes
X = X.drop(columns=['transaction_id', 'customer_id', 'device_id'])


In [ ]:
print(X.dtypes)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Liste des colonnes catégorielles à encoder
cols_to_encode = ['transaction_type', 'location', 'status', ]

# Encoder les colonnes en valeurs numériques
label_encoder = LabelEncoder()
for col in cols_to_encode:
    X[col] = label_encoder.fit_transform(X[col].astype(str))

# Vérifier les changements
print(X.dtypes)



In [ ]:
# Étape 3: Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# `test_size=0.2` signifie que 20% des données seront utilisées pour le test, 80% pour l'entraînement
# `random_state=42` est un paramètre pour reproduire les résultats aléatoires

In [ ]:
# Créer et entraîner le modèle RandomForest
modele = RandomForestClassifier(n_estimators=100, random_state=42)
modele.fit(X_train, y_train)  # Entraînement du modèle sur les données d’entraînement


In [ ]:
y_pred = modele.predict(X_test)  # Prédiction des données de test


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

# Calculer la précision
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision (Accuracy) : {accuracy:.4f}")

# Afficher la matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matrice de confusion :\n", conf_matrix)

# Afficher un rapport de classification détaillé (précision, rappel, F1-score)
print("Rapport de classification :\n", classification_report(y_test, y_pred))

# Calculer le score F1
f1 = f1_score(y_test, y_pred)
print(f"Score F1 : {f1:.4f}")


# INTERPRETATION: notre modele a  62,63 % de predictions correctes

# LE BAGGING #

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
# CREATION DE NOTRE MODELE DE BAGGING

# Définissons le modèle de base (RandomForestClassifier)
base_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=15)

# Création du BaggingClassifier avec notre modèle de base
bagging_model = BaggingClassifier(estimator=base_model, 
                                  n_estimators=10,   # Nombre de modèles à entraîner
                                  random_state=42,
                                  n_jobs=-1)  # Utilisation de tous les cœurs CPU disponibles


In [ ]:
# Entraînons le modèle  sur nos données d'entraînement
bagging_model.fit(X_train, y_train)


In [ ]:
#  prédictions sur les données de test
y_pred = bagging_model.predict(X_test)

# Calcul de la précision (Accuracy)
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision (Accuracy) : {accuracy:.4f}")

#  matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matrice de confusion :")
print(conf_matrix)

# rapport de classification
class_report = classification_report(y_test, y_pred)
print("Rapport de classification :")
print(class_report)


# INTERPRETATION: notre modele a  65,38 % de predictions correctes, soit 2.75% de plus que notre modele de classification.

# LE BOOSTING 

## RECHERCHE DES MEILLEURS HYPERPARAMETRES ##

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from scipy.stats import uniform, randint

# Définir la distribution des paramètres à tester
param_dist = {
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'n_estimators': randint(100, 500),
    'subsample': uniform(0.7, 0.3),
    'colsample_bytree': uniform(0.7, 0.3),
    'gamma': uniform(0, 0.5)
}

# Créer le modèle XGBoost
xgb_model = xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False)

# Configurer RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, n_iter=50, cv=3, scoring='f1', verbose=1, n_jobs=-1, random_state=42)

# Exécuter la recherche aléatoire
random_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres
print(f"Meilleurs paramètres trouvés : {random_search.best_params_}")


In [ ]:

# Créer un modèle XGBoost avec les meilleurs hyperparamètres trouvés

best_xgb_model = xgb.XGBClassifier(
    colsample_bytree=0.7816396748153905,
    gamma=0.32384506027068116,
    learning_rate=0.010156113098594747,
    max_depth=7,
    n_estimators=332,
    subsample=0.7914343774474086,
    objective='binary:logistic',
    use_label_encoder=False,
    random_state=42
)

# Entraîner le modèle sur les données d'entraînement
best_xgb_model.fit(X_train, y_train)

# Prédire les étiquettes pour les données de test
y_pred = best_xgb_model.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Précision (Accuracy): {accuracy:.4f}")
print(f"Score F1: {f1:.4f}")
print("Matrice de confusion :\n", confusion)
print("Rapport de classification :\n", classification_rep)


# la precision de notre modele de boosting  est de 66,29%, soit 1% plus precis que le bagging. 

### Cherchons a comprendre si c'est la technique qui est meilleure pour notre cas ou alors si c'est grace à l'utilisation des meilleurrs hyperparametres que nous avons de meilleurs resultats.

# BAGGING AVEC LES HYPERPARAMETRES TROUVES 

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# Créer le classifieur de base avec les hyperparamètres optimisés
base_classifier = DecisionTreeClassifier(
    max_depth=7,
    random_state=42
)

# Créer le modèle Bagging avec les paramètres optimisés
bagging_model = BaggingClassifier(
    estimator=base_classifier,
    n_estimators=332,  # équivalent à n_estimators dans XGBoost
    max_samples=0.7914343774474086,  # équivalent à subsample dans XGBoost
    random_state=42
)

# Entraîner le modèle sur les données d'entraînement
bagging_model.fit(X_train, y_train)

# Prédire les étiquettes pour les données de test
y_pred_bagging = bagging_model.predict(X_test)

# Évaluer les performances du modèle
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
f1_bagging = f1_score(y_test, y_pred_bagging)
confusion_bagging = confusion_matrix(y_test, y_pred_bagging)
classification_rep_bagging = classification_report(y_test, y_pred_bagging)

print(f"Précision (Accuracy): {accuracy_bagging:.4f}")
print(f"Score F1: {f1_bagging:.4f}")
print("Matrice de confusion :\n", confusion_bagging)
print("Rapport de classification :\n", classification_rep_bagging)


# INTERPRETATION #

## PREDICTIONS CORRECTES: 66,36%

Vrais négatifs (TN) : 5661 / non fraudes classées comme non fraudes
Faux positifs (FP) : 5148  / non fraudes classées comme fraude
Faux négatifs (FN) : 3263 / fraudes classés comme non fraudes
Vrais positifs (TP) : 10 928 / fraudes classées comme fraude
## FRAUDES DETECTEES: 10 928 et FRAUDES NON DETECTEES: 3263

63% de prediction correctes sur la classe de non fraude et 68% de predictions correctes sur la classe des fraudes 
## (le f1 score nous montre que notre modele a mieux detecter les cas de fraudes que de non fraude)

# conclusion: 

### l'utilisaton des meilleurs hyperparametres couplé avec un arbre de decision comme modele de base a amelioré la precision de notre modele de Bagging de 1% par rapport au modele de bagging utilisant des hyperparametres aleatoires .  

### En comparant les performances de nos methodes (bagging et boosting) couplés avec l'utilisation des hyperparametres trouvés, on se rend compte que les resultats en utilisant le bagging sont pratiquement egales(0.07% de difference). 

# On conclut que utiliser un modèle d'ensemble a amélioré nos resultats. Cependant, le choix parmi les deux modèles d'ensemble a eu peu d'influence sur nos resultats mais utiliser les meilleurs hyperparametres a ete le facteur ayant le plus amelioré les resultats de nos 2 modèles.

In [ ]:
import joblib

# Entraînement du modèle (par exemple, un modèle RandomForest)
# modèle = RandomForestClassifier(...)
# modèle.fit(X_train, y_train)

# Enregistrer le modèle entraîné
joblib.dump(bagging_model, 'bagging_model.pkl')

print("Le modèle a été enregistré avec succès.")


In [ ]:
# Charger le modèle entraîné
# bagging_model_reloaded = joblib.load('bagging_model.pkl')

